In [ ]:
#collect paddlex imblearn
!pip install paddlex==2.0.0rc4 -i https://mirror.baidu.com/pypi/simple
!pip install imblearn

Looking in indexes: https://mirror.baidu.com/pypi/simple
     |████████████████████████████████| 993kB 13.0MB/s eta 0:00:01
     |████████████████████████████████| 307kB 13.9MB/s eta 0:00:01
     |████████████████████████████████| 153kB 23.7MB/s eta 0:00:01
     |████████████████████████████████| 1.5MB 12.8MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 13.8MB/s eta 0:00:01
     |████████████████████████████████| 6.8MB 16.1MB/s eta 0:00:01
     |████████████████████████████████| 286kB 23.4MB/s eta 0:00:01
     |████████████████████████████████| 51kB 14.7MB/s eta 0:00:01
     |████████████████████████████████| 102kB 11.9MB/s ta 0:00:01
     |████████████████████████████████| 102kB 23.7MB/s ta 0:00:01
  Created wheel for lap: filename=lap-0.4.0-cp37-cp37m-linux_x86_64.whl size=1593863 sha256=d1e9fd76782e552627c589ec11e30766e3016d5178ce872f2300c60e850728d3
  Stored in directory: /home/aistudio/.cache/pip/wheels/83/50/a9/e3660736bfb1fb50598b822551bb8c7ff04f1a4ecf69c42277
  C

# paddlex

In [ ]:
!unzip data/data105081/test.zip -d work
!unzip data/data105081/train.zip -d work

In [ ]:
import matplotlib.pyplot as plt
plt.style.use('seaborn')                   # if want to use the default style, set 'classic'
plt.rcParams['ytick.right']     = True
plt.rcParams['ytick.labelright']= True
plt.rcParams['ytick.left']      = False
plt.rcParams['ytick.labelleft'] = False
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import paddlex as pdx
from paddlex import transforms as T
from collections import Counter

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:120: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."


[09-20 13:38:25 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_context()
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/pyarrow/pandas_compat.py:1027: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  'floating': np.float,
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddleslim/common/analyze_helper.py:22: UserWarning: 
This call to matplotlib.use() has no effect because

In [ ]:
#split dataset
!paddlex --split_dataset --format ImageNet --dataset_dir work/train --val_value 0.1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/distributed/parallel.py:120: UserWarning: Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything.
  "Currently not a parallel execution environment, `paddle.distributed.init_parallel_env` will not do anything."
[09-20 13:38:29 MainThread @logger.py:242] Argv: /opt/conda/envs/python35-paddle120-env/bin/paddlex --split_dataset --format ImageNet --dataset_dir work/train --val_value 0.1
[09-20 13:38:29 MainThread @utils.py:79] WRN paddlepaddle version: 2.1.2. The dynamic graph version of PARL is under development, not fully tested and supported
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/parl/remote/communication.py:38: DeprecationWarning: 'pyarrow.default_serialization_context' is deprecated as of 2.0.0 and will be removed in a future version. Use pickle or the pyarrow IPC functionality instead.
  context = pyarrow.default_serialization_cont

In [ ]:
# data augument
train_transforms = T.Compose([
                            T.Resize(target_size=(128,128)),
                            T.RandomBlur(), 
                            T.RandomHorizontalFlip(), 
                            T.MixupImage(),
                            T.RandomDistort(),
                            T.Normalize(),
                            ])                        
eval_transforms = T.Compose([T.Resize(target_size=(128,128)),T.Normalize()])

In [ ]:
train_dataset = pdx.datasets.ImageNet(
    data_dir='work/train',
    file_list='work/train/train_list.txt',
    label_list='work/train/labels.txt',
    transforms=train_transforms,
    shuffle=True)
print(len(train_dataset)) 
eval_dataset = pdx.datasets.ImageNet(
    data_dir='work/train',
    file_list='work/train/val_list.txt',
    label_list='work/train/labels.txt',
    transforms=eval_transforms)
print(len(eval_dataset))

2021-09-20 13:38:34 [INFO]	Starting to read file list from dataset...
2021-09-20 13:38:34 [INFO]	25841 samples in file work/train/train_list.txt
25841
2021-09-20 13:38:34 [INFO]	Starting to read file list from dataset...
2021-09-20 13:38:34 [INFO]	2868 samples in file work/train/val_list.txt
2868


In [ ]:
num_classes = len(train_dataset.labels)
model = pdx.cls.MobileNetV3_large_ssld(num_classes=num_classes)
model.train(num_epochs=20,
            train_dataset=train_dataset,
            train_batch_size=16,
            eval_dataset=eval_dataset,
            lr_decay_epochs=[10, 15, 17],
            save_interval_epochs=1,
            learning_rate=0.0025,
            early_stop=True,
            save_dir='output/MobileNetV3_large_ssld',
            use_vdl=True)

2021-09-20 13:38:59 [INFO]	Downloading MobileNetV3_large_x1_0_ssld_pretrained.pdparams from https://paddle-imagenet-models-name.bj.bcebos.com/dygraph/MobileNetV3_large_x1_0_ssld_pretrained.pdparams


100%|██████████| 32444/32444 [00:00<00:00, 37128.78KB/s]


In [ ]:
import pandas as pd
test=pd.read_csv('work/sample_submit.csv')
model = pdx.load_model('output/DarkNet53/best_model/')
labels=[]
for index, item in test.iterrows():  
    image_name = os.path.join('work/test', item['name'])
    label = model.predict(image_name)
    labels.append(label)
print("Predict Done:", len(labels))

2021-09-20 11:28:30 [INFO]	Model[DarkNet53] loaded.


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


Predict Done: 7178


In [ ]:
test['label']=labels
for i in range(len(labels)):
    test['label'][i]=test['label'][i][0]['category']

# 不要index
test.to_csv('result.csv', index=False)



In [ ]:
!visualdl --logdir output/ 

VisualDL 2.2.0
Running VisualDL at http://localhost:8040/ (Press CTRL+C to quit)
Serving VisualDL on localhost; to expose to the network, use a proxy or pass --host 0.0.0.0
^C


In [ ]:
#just test 1 image
import paddlex as pdx
test_jpg = 'work/train/fearful/im183.png'
model = pdx.load_model('best_model/ResNet101_cd_ssld/model/')
result = model.predict(test_jpg)
print("Predict Result: ", result[0]['category'])

2021-09-20 14:45:19 [INFO]	Model[ResNet101_vd_ssld] loaded.
Predict Result:  fearful


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


In [28]:
import pandas as pd

model=[]
model.append(pdx.load_model('best_model/ResNet101_cd_ssld/model/'))
model.append(pdx.load_model('best_model/Xception71/model/'))
model.append(pdx.load_model('best_model/DenseNet264/model/'))
model.append(pdx.load_model('best_model/DarkNet53/model/'))
model.append(pdx.load_model('best_model/ShuffleNetV2_swish/model/'))
model.append(pdx.load_model('best_model/MobileNetV3_large_ssld/model/'))
print(model[0].predict(test_jpg))

test=pd.read_csv('work/sample_submit.csv')

labels=[]
images_namels=[]
for index, item in test.iterrows():  
    image_name = os.path.join('work/test', item['name'])
    images_namels.append(image_name)

2021-09-20 14:56:55 [INFO]	Model[ResNet101_vd_ssld] loaded.
2021-09-20 14:56:55 [INFO]	Model[Xception71] loaded.
2021-09-20 14:56:56 [INFO]	Model[DenseNet264] loaded.
2021-09-20 14:56:57 [INFO]	Model[DarkNet53] loaded.
2021-09-20 14:56:57 [INFO]	Model[ShuffleNetV2_swish] loaded.
2021-09-20 14:56:57 [INFO]	Model[MobileNetV3_large_ssld] loaded.


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


[{'category_id': 2, 'category': 'fearful', 'score': 0.98979795}]


In [29]:
for img in images_namels:
    probs=0
    best_model=None
    for i in range(0,6):
        model_test=model[i]
        result=model_test.predict(img)[0]
        tmp_prob=result['score']
        if(tmp_prob>probs):
            probs=tmp_prob
            label=result['category']
    labels.append(label)

print("Predict Done:", len(labels))

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if data.dtype == np.object:


In [ ]:
test=pd.read_csv('work/sample_submit.csv')
test['label']=labels

# 不要index
test.to_csv('result.csv', index=False)